In [49]:
# 데이터를 표준화(스케일링), 분할  28 28 1
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()
# 데이터 로드
(x_train,y_train),(x_test,y_test) =  keras.datasets.fashion_mnist.load_data()
ss = StandardScaler()
x_train_scaled =  ss.fit_transform(x_train.reshape( x_train.shape[0],-1 ))
print(x_train_scaled.shape, y_train.shape)
x_train,x_test,y_train,y_test =  train_test_split(x_train_scaled,y_train,random_state=32)
print(x_train.shape, y_train.shape)
x_train_3d = x_train.reshape(-1,28,28,1)
x_train_3d.shape

(60000, 784) (60000,)
(45000, 784) (45000,)


(45000, 28, 28, 1)

In [50]:
model = keras.Sequential()

In [51]:
# 1번째 합성곱 층
model.add(keras.layers.Conv2D(32, kernel_size=3, activation='relu', padding='same',input_shape=(28,28,1)) )
# (28,28,32) 특성맵
model.add(keras.layers.MaxPooling2D(2))  # (14,14,32)

In [52]:
#(14,14,32)

In [53]:
# 2번째 합성곱 층
model.add(keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu',padding='same'))
# 특성맵 (14,14,64)
model.add(keras.layers.MaxPooling2D(2))
# 특성맵(7,7,64)

In [54]:
# 완전연결층에 앞에서만든 합성곱을 연결한다.
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(10,activation='softmax'))


In [55]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 3136)              0         
                                                                 
 dense_6 (Dense)             (None, 100)              

In [56]:
!pip install pydot

In [57]:
!pip install graphviz

In [58]:
from tensorflow.keras.utils import plot_model

In [59]:
plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [60]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

In [61]:
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-cnn-model.h5',save_best_only=True)

In [62]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2,restore_best_weights=True)

In [63]:
history = model.fit(x_train_3d,y_train,epochs=100,validation_data=(x_test,y_test)
                    , callbacks=[checkpoint_cb,early_stopping_cb] )

Epoch 1/100
1406/1407 [============================>.] - ETA: 0s - loss: 0.4908 - accuracy: 0.8279

ValueError: in user code:

    File "C:\Users\user\miniconda3\envs\ml\lib\site-packages\keras\engine\training.py", line 1557, in test_function  *
        return step_function(self, iterator)
    File "C:\Users\user\miniconda3\envs\ml\lib\site-packages\keras\engine\training.py", line 1546, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\user\miniconda3\envs\ml\lib\site-packages\keras\engine\training.py", line 1535, in run_step  **
        outputs = model.test_step(data)
    File "C:\Users\user\miniconda3\envs\ml\lib\site-packages\keras\engine\training.py", line 1499, in test_step
        y_pred = self(x, training=False)
    File "C:\Users\user\miniconda3\envs\ml\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\user\miniconda3\envs\ml\lib\site-packages\keras\engine\input_spec.py", line 264, in assert_input_compatibility
     